In [ ]:
train = pd.read_csv(r'C:/Users/cny13/OneDrive/바탕 화면/빅콘/segmentation용 데이터/train_notna_newlog2.csv')   

In [ ]:
train2 = train.copy()

In [ ]:
#형 변환
train2['loanapply_insert_time'] = pd.to_datetime(train2['loanapply_insert_time'])
train2['insert_time'] = pd.to_datetime(train2['insert_time'])

In [ ]:
# 변수 값 변경
train2.loc[train2['gender'] == 1, 'gender'] = 'male'
train2.loc[train2['gender'] == 0, 'gender'] = 'female'

In [ ]:
# 필요없는 변수 삭제
cols = ['대출총괄_mo','대출총괄_pc','주택관련대출_mo','주택관련대출_pc','자동차대출_mo','자동차대출_pc',
        '대환대출_mo','대환대출_pc','핀다대출외서비스_mo','핀다대출외서비스_pc','CPI','CPI_lag1','cofix']
train2 = train2.drop(cols,axis=1)

In [ ]:
train2 = train2.drop('index', axis = 1)

In [ ]:
cols = ['before_7days','before_30days','insert_time_weekday_sin','insert_time_weekday_cos']
train2 = train2.drop(cols,axis=1)

In [ ]:
pd.set_option('display.max_columns',None)
train2.head(5)

In [ ]:
train2.shape

# 기존 로그데이터 삭제

In [ ]:
lst = list(train2.columns[27:40])
train2 = train2.drop(lst,axis=1)

In [ ]:
lst = list(train2.columns[48:61])
train2 = train2.drop(lst,axis=1)

In [ ]:
train2 = train2.drop('mp_os',axis=1)

In [ ]:
train2.head()

In [ ]:
train2.shape

In [ ]:
train2.to_csv('C:/Users/cny13/OneDrive/바탕 화면/빅콘/segmentation용 데이터/lightsegmentation.csv',index=False)

# 여기부터 시작~!

In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv(r'C:/Users/cny13/OneDrive/바탕 화면/빅콘/segmentation용 데이터/lightsegmentation.csv')   

In [3]:
train['loanapply_insert_time'] = pd.to_datetime(train['loanapply_insert_time'])
train['insert_time'] = pd.to_datetime(train['insert_time'])

In [4]:
train2 = train.copy()

In [5]:
train2.user_id.nunique()

162900

# log 데이터 한줄로 만들기

In [6]:
newlog = pd.read_csv(r'C:/Users/cny13/OneDrive/바탕 화면/빅콘/segmentation용 데이터/customer_log_ver2.csv')

In [7]:
#frequency2 =유저별 visit_rate max
newlog['frequency2'] = newlog.groupby('user_id')['visit_rate'].transform(max)
newlog.head(5)

,user_id,date_cd,mp_os,Session,how_long,visit_cumsum,visit_rate,OpenApp,Login,StartLoanApply,ViewLoanApplyIntro,CompleteIDCertification,EndLoanApply,UseDSRCalc,UseLoanManage,UsePrepayCalc,GetCreditInfo,frequency2
0,1,2022-05-03,unknown,1,1,0.0,0.000000,1,1,0,0,0,0,0,0,0,2,0.000000
1,7,2022-05-22,unknown,1,1,0.0,0.000000,1,1,0,0,0,0,0,0,0,1,0.000000
2,9,2022-05-21,unknown,1,1,0.0,0.000000,1,1,0,0,0,0,0,0,0,3,0.000000
3,11,2022-03-24,iOS,1,1,0.0,0.000000,1,1,1,1,1,1,0,2,1,3,0.035714
4,11,2022-04-20,iOS,1,28,1.0,0.035714,2,1,4,4,4,4,0,5,0,5,0.035714


In [8]:
#행동 sum 변수 만들기
newlog['CompleteIDCertification_sum'] = newlog.groupby('user_id')['CompleteIDCertification'].transform(sum)
newlog['EndLoanApply_sum'] = newlog.groupby('user_id')['EndLoanApply'].transform(sum)
newlog['GetCreditInfo_sum'] = newlog.groupby('user_id')['GetCreditInfo'].transform(sum)
newlog['Login_sum'] = newlog.groupby('user_id')['Login'].transform(sum)
newlog['OpenApp_sum'] = newlog.groupby('user_id')['OpenApp'].transform(sum)
newlog['StartLoanApply_sum'] = newlog.groupby('user_id')['StartLoanApply'].transform(sum)
newlog['UseDSRCalc_sum'] = newlog.groupby('user_id')['UseDSRCalc'].transform(sum)
newlog['UseLoanManage_sum'] = newlog.groupby('user_id')['UseLoanManage'].transform(sum)
newlog['UsePrepayCalc_sum'] = newlog.groupby('user_id')['UsePrepayCalc'].transform(sum)
newlog['ViewLoanApplyIntro_sum'] = newlog.groupby('user_id')['ViewLoanApplyIntro'].transform(sum)


In [9]:
cols4 = ['CompleteIDCertification','EndLoanApply','GetCreditInfo','Login','OpenApp','StartLoanApply',
        'UseDSRCalc','UseLoanManage','UsePrepayCalc','ViewLoanApplyIntro']
newlog = newlog.drop(cols4,axis=1)

In [10]:
#how_long_total : 총 가입일수
newlog['how_long_total'] = newlog.groupby('user_id')['how_long'].transform(max)

In [11]:
# managing_service 변수 만들기
lst = ['user_id','UseDSRCalc_sum','UseLoanManage_sum','UsePrepayCalc_sum','GetCreditInfo_sum']
df = newlog[lst]
df = df.drop_duplicates('user_id')

df2 = pd.DataFrame({'user_id': df.user_id,
                   'managing_service':np.nan})
lst2 = ['UseDSRCalc_sum','UseLoanManage_sum','UsePrepayCalc_sum','GetCreditInfo_sum']

df = df[lst2]

df2['managing_service'] = df.sum(axis = 1)
newlog = pd.merge(newlog, df2, on='user_id', how= 'left')

In [12]:
newlog.head()

,user_id,date_cd,mp_os,Session,how_long,visit_cumsum,visit_rate,frequency2,CompleteIDCertification_sum,EndLoanApply_sum,GetCreditInfo_sum,Login_sum,OpenApp_sum,StartLoanApply_sum,UseDSRCalc_sum,UseLoanManage_sum,UsePrepayCalc_sum,ViewLoanApplyIntro_sum,how_long_total,managing_service
0,1,2022-05-03,unknown,1,1,0.0,0.000000,0.000000,0,0,2,1,1,0,0,0,0,0,1,2
1,7,2022-05-22,unknown,1,1,0.0,0.000000,0.000000,0,0,1,1,1,0,0,0,0,0,1,1
2,9,2022-05-21,unknown,1,1,0.0,0.000000,0.000000,0,0,3,1,1,0,0,0,0,0,1,3
3,11,2022-03-24,iOS,1,1,0.0,0.000000,0.035714,5,5,8,2,3,5,0,7,1,5,28,16
4,11,2022-04-20,iOS,1,28,1.0,0.035714,0.035714,5,5,8,2,3,5,0,7,1,5,28,16


In [13]:
print(newlog.shape)
print(newlog.user_id.nunique())

(2507971, 20)
479773


In [14]:
# 유저별로 마지막 줄만 남기기
newlog = newlog.drop_duplicates(['user_id'], keep = 'last')
print(newlog.shape)
print(newlog.user_id.nunique())

(479773, 20)
479773


In [15]:
pd.set_option('display.max_columns',None) #visit_rate -> 유저별 마지막 방문빈도
newlog.head()

,user_id,date_cd,mp_os,Session,how_long,visit_cumsum,visit_rate,frequency2,CompleteIDCertification_sum,EndLoanApply_sum,GetCreditInfo_sum,Login_sum,OpenApp_sum,StartLoanApply_sum,UseDSRCalc_sum,UseLoanManage_sum,UsePrepayCalc_sum,ViewLoanApplyIntro_sum,how_long_total,managing_service
0,1,2022-05-03,unknown,1,1,0.0,0.000000,0.000000,0,0,2,1,1,0,0,0,0,0,1,2
1,7,2022-05-22,unknown,1,1,0.0,0.000000,0.000000,0,0,1,1,1,0,0,0,0,0,1,1
2,9,2022-05-21,unknown,1,1,0.0,0.000000,0.000000,0,0,3,1,1,0,0,0,0,0,1,3
4,11,2022-04-20,iOS,1,28,1.0,0.035714,0.035714,5,5,8,2,3,5,0,7,1,5,28,16
15,12,2022-05-19,Android,1,67,9.0,0.134328,0.134328,24,24,3,11,11,25,0,1,0,25,67,4


# 로그 데이터 병합

In [16]:
users = list(train2['user_id'].unique())
len(users)

162900

In [17]:
temp = newlog.loc[newlog['user_id'].isin(users)]
print('로그1:',newlog.user_id.nunique())
print('로그2:',temp.user_id.nunique())
print('train:',train2.user_id.nunique()) 


로그1: 479773
로그2: 161706
train: 162900


In [18]:
temp.rename(columns = {'date_cd' : 'date'}, inplace = True)

C:\Users\cny13\anaconda3\envs\test\lib\site-packages\pandas\core\frame.py:5042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
temp.head()

,user_id,date,mp_os,Session,how_long,visit_cumsum,visit_rate,frequency2,CompleteIDCertification_sum,EndLoanApply_sum,GetCreditInfo_sum,Login_sum,OpenApp_sum,StartLoanApply_sum,UseDSRCalc_sum,UseLoanManage_sum,UsePrepayCalc_sum,ViewLoanApplyIntro_sum,how_long_total,managing_service
2,9,2022-05-21,unknown,1,1,0.0,0.000000,0.000000,0,0,3,1,1,0,0,0,0,0,1,3
4,11,2022-04-20,iOS,1,28,1.0,0.035714,0.035714,5,5,8,2,3,5,0,7,1,5,28,16
39,24,2022-05-16,Android,2,4,1.0,0.250000,0.250000,2,2,4,3,3,2,0,1,0,2,4,5
58,27,2022-05-19,Android,2,48,7.0,0.145833,0.333333,5,5,12,14,15,5,0,10,0,5,48,22
78,31,2022-04-18,Android,1,39,2.0,0.051282,0.051282,3,3,6,5,5,3,0,4,0,3,39,10


In [20]:
train3 = pd.merge(train2, temp, how = 'left', on=['user_id','date'])
train3.shape

(7733096, 67)

In [21]:
print(train2.shape)
print(train3.shape)

(7733096, 49)
(7733096, 67)


In [22]:
train3 = train3.dropna(axis = 0)

In [23]:
train3.shape

(1512558, 67)

In [24]:
train3.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,company_enter_year,age,Session_x,insert_time_month,insert_time_day,insert_time_hour,insert_time_weekday,insert_time_timezone,company_enter_month_year,company_enter_month_month,is_holiday,insert_time_diff,application_id_cnt,diff_amount,more_loan_desired,loan_limit_outlier,first_confirmed,bank_cumcnt,bank_popularity,product_cumcnt,product_popularity,bank_group,product_group,date,loanapply_insert_time_s,apply_cnt,mp_os,Session_y,how_long,visit_cumsum,visit_rate,frequency2,CompleteIDCertification_sum,EndLoanApply_sum,GetCreditInfo_sum,Login_sum,OpenApp_sum,StartLoanApply_sum,UseDSRCalc_sum,UseLoanManage_sum,UsePrepayCalc_sum,ViewLoanApplyIntro_sum,how_long_total,managing_service
0,248576,2022-05-21 23:41:09,27,148,16000000.0,7.0,0,9,1996.0,male,2022-05-21 23:41:09,810.0,29000000.0,EARNEDINCOME,202201.0,계약직,기타가족소유,15000000.0,BUSINESS,0.0,no_rehabilitation,1.0,6.299001e+07,2022.0,27.0,1.0,5,21,23,5,evening,2022,1,False,0.0,26,-1000000.0,True,False,True,196689,0.073166,78581,0.080897,2,2,2022-05-21,7170069,0,unknown,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
1,248576,2022-05-21 23:41:09,59,251,16000000.0,6.5,0,9,1996.0,male,2022-05-21 23:41:09,810.0,29000000.0,EARNEDINCOME,202201.0,계약직,기타가족소유,15000000.0,BUSINESS,0.0,no_rehabilitation,1.0,6.299001e+07,2022.0,27.0,1.0,5,21,23,5,evening,2022,1,False,0.0,26,-1000000.0,True,False,False,272147,0.107097,108594,0.183307,5,2,2022-05-21,7170069,0,unknown,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
2,248576,2022-05-21 23:41:09,59,150,20000000.0,15.0,0,9,1996.0,male,2022-05-21 23:41:09,810.0,29000000.0,EARNEDINCOME,202201.0,계약직,기타가족소유,15000000.0,BUSINESS,0.0,no_rehabilitation,1.0,6.299001e+07,2022.0,27.0,1.0,5,21,23,5,evening,2022,1,False,0.0,26,-5000000.0,True,False,False,272146,0.107097,163553,0.056495,5,6,2022-05-21,7170069,0,unknown,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
3,248576,2022-05-21 23:41:09,38,134,16000000.0,7.5,0,9,1996.0,male,2022-05-21 23:41:09,810.0,29000000.0,EARNEDINCOME,202201.0,계약직,기타가족소유,15000000.0,BUSINESS,0.0,no_rehabilitation,1.0,6.299001e+07,2022.0,27.0,1.0,5,21,23,5,evening,2022,1,False,0.0,26,-1000000.0,True,False,False,341164,0.047986,85179,0.070804,5,2,2022-05-21,7170069,0,unknown,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
4,248576,2022-05-21 23:41:10,55,25,16000000.0,14.5,0,9,1996.0,male,2022-05-21 23:41:09,810.0,29000000.0,EARNEDINCOME,202201.0,계약직,기타가족소유,15000000.0,BUSINESS,0.0,no_rehabilitation,1.0,6.299001e+07,2022.0,27.0,1.0,5,21,23,5,evening,2022,1,False,1.0,26,-1000000.0,True,False,False,110711,0.012483,36466,0.010448,5,7,2022-05-21,7170070,0,unknown,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0


# RFM 만들기

### 1) frequency: 가입기간 대비 얼마나 자주 apply=1 했나.
총 apply=1인 개수  / 가입일수

 = 총 1인 apply 개수/ 유저별 max how_long

In [25]:
train3['apply_cnt'] = train3.groupby('user_id')['is_applied'].transform(sum)

In [26]:
train3['frequency'] = train3['apply_cnt']/train3['how_long_total']

### 2) recency: 마지막 신청일시로부터 6/1까지 며칠인지

In [27]:
#recency 
temp_recency_df = train3.drop_duplicates(subset ='user_id',keep='last')[['user_id','loanapply_insert_time']]
now = pd.to_datetime('2022-06-01')
temp_recency_df['recency'] = (now - temp_recency_df['loanapply_insert_time']).dt.days
recency_df = temp_recency_df[['user_id','recency']]
recency_df.shape

(82376, 2)

In [28]:
train3 = pd.merge(train3, recency_df, on ='user_id', how = 'left')
train3 = train3.sort_values('loanapply_insert_time')
train3.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,company_enter_year,age,Session_x,insert_time_month,insert_time_day,insert_time_hour,insert_time_weekday,insert_time_timezone,company_enter_month_year,company_enter_month_month,is_holiday,insert_time_diff,application_id_cnt,diff_amount,more_loan_desired,loan_limit_outlier,first_confirmed,bank_cumcnt,bank_popularity,product_cumcnt,product_popularity,bank_group,product_group,date,loanapply_insert_time_s,apply_cnt,mp_os,Session_y,how_long,visit_cumsum,visit_rate,frequency2,CompleteIDCertification_sum,EndLoanApply_sum,GetCreditInfo_sum,Login_sum,OpenApp_sum,StartLoanApply_sum,UseDSRCalc_sum,UseLoanManage_sum,UsePrepayCalc_sum,ViewLoanApplyIntro_sum,how_long_total,managing_service,frequency,recency
574866,584435,2022-03-01 00:15:22,45,119,17000000.0,3.6,1,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,0.0,11,83000000.0,False,False,False,4,0.500000,3,0.666667,2,2,2022-03-01,87322,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91
574867,584435,2022-03-01 00:15:22,13,123,10000000.0,19.1,0,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,0.0,11,90000000.0,False,False,False,8,0.000000,3,0.000000,4,6,2022-03-01,87322,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91
574868,584435,2022-03-01 00:15:22,24,263,5000000.0,15.3,0,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,0.0,11,95000000.0,False,False,False,5,0.000000,3,0.000000,5,7,2022-03-01,87322,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91
574869,584435,2022-03-01 00:15:22,30,85,10000000.0,16.6,0,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,0.0,11,90000000.0,False,False,True,6,0.166667,3,0.333333,5,6,2022-03-01,87322,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91
574874,584435,2022-03-01 00:15:23,52,234,41000000.0,5.6,1,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,1.0,11,59000000.0,False,False,False,3,0.333333,2,0.500000,2,2,2022-03-01,87323,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91


In [ ]:
### 3) monetary: 유저별 총신청금액
# yes_apply = train3.loc[train3['is_applied']==1]
# no_apply = train3.loc[train3['is_applied']==0]

In [ ]:
# yes_apply['loanapply_insert_time'] = pd.to_datetime(yes_apply['loanapply_insert_time'])
# yes_apply['date'] = yes_apply['loanapply_insert_time'].dt.date
# yes_apply = yes_apply.sort_values('loanapply_insert_time')
# yes_apply['monetary'] = yes_apply.groupby(['user_id','date'])['desired_amount'].transform('mean') 
# yes_apply['monetary'] = yes_apply.groupby('user_id')['monetary'].transform('sum')

In [ ]:
# monetary_df = yes_apply[['user_id','monetary']]

In [ ]:
# monetary_df = monetary_df.drop_duplicates('user_id')

In [ ]:
# train3 = pd.merge(train3, monetary_df, on='user_id', how='left')

In [ ]:
# train3.fillna(0, inplace =True)

In [ ]:
# train3.isna().sum().sum()

# FRM2 만들기

### recency2 : 마지막 접속일로부터 6/1까지 며칠 지났는지
6/1 - 'date' 변수

In [32]:
train3['date'] = pd.to_datetime(train3['date'])

In [34]:
#recency 
temp_recency_df = train3.drop_duplicates(subset ='user_id',keep='last')[['user_id','date']]
now = pd.to_datetime('2022-06-01')
temp_recency_df['recency2'] = (now - temp_recency_df['date']).dt.days
recency_df = temp_recency_df[['user_id','recency2']]
recency_df.shape


(82376, 2)

In [35]:
train3 = pd.merge(train3, recency_df, on ='user_id', how = 'left')
train3.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,company_enter_year,age,Session_x,insert_time_month,insert_time_day,insert_time_hour,insert_time_weekday,insert_time_timezone,company_enter_month_year,company_enter_month_month,is_holiday,insert_time_diff,application_id_cnt,diff_amount,more_loan_desired,loan_limit_outlier,first_confirmed,bank_cumcnt,bank_popularity,product_cumcnt,product_popularity,bank_group,product_group,date,loanapply_insert_time_s,apply_cnt,mp_os,Session_y,how_long,visit_cumsum,visit_rate,frequency2,CompleteIDCertification_sum,EndLoanApply_sum,GetCreditInfo_sum,Login_sum,OpenApp_sum,StartLoanApply_sum,UseDSRCalc_sum,UseLoanManage_sum,UsePrepayCalc_sum,ViewLoanApplyIntro_sum,how_long_total,managing_service,frequency,recency,recency2
0,584435,2022-03-01 00:15:22,45,119,17000000.0,3.6,1,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,0.0,11,83000000.0,False,False,False,4,0.500000,3,0.666667,2,2,2022-03-01,87322,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91,92
1,584435,2022-03-01 00:15:22,13,123,10000000.0,19.1,0,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,0.0,11,90000000.0,False,False,False,8,0.000000,3,0.000000,4,6,2022-03-01,87322,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91,92
2,584435,2022-03-01 00:15:22,24,263,5000000.0,15.3,0,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,0.0,11,95000000.0,False,False,False,5,0.000000,3,0.000000,5,7,2022-03-01,87322,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91,92
3,584435,2022-03-01 00:15:22,30,85,10000000.0,16.6,0,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,0.0,11,90000000.0,False,False,True,6,0.166667,3,0.333333,5,6,2022-03-01,87322,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91,92
4,584435,2022-03-01 00:15:23,52,234,41000000.0,5.6,1,334783,1984.0,male,2022-03-01 00:15:22,830.0,60000000.0,PRIVATEBUSINESS,201608.0,정규직,기타가족소유,100000000.0,BUSINESS,no_response,no_response,1.0,6.299001e+07,2016.0,39.0,1.0,3,1,0,1,dawn_not1,2016,8,True,1.0,11,59000000.0,False,False,False,3,0.333333,2,0.500000,2,2,2022-03-01,87323,2,Android,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,91,92


In [ ]:
# #총 활동횟수(monetary2)
# lst = ['user_id','CompleteIDCertification_sum','EndLoanApply_sum','GetCreditInfo_sum','Login_sum','OpenApp_sum',
#       'StartLoanApply_sum','UseDSRCalc_sum','UseLoanManage_sum','UsePrepayCalc_sum','ViewLoanApplyIntro_sum']
# df = train3[lst]
# df = df.drop_duplicates('user_id')
# df.head(5)

In [ ]:
# import numpy as np
# df2 = pd.DataFrame({'user_id': df.user_id,
#                    'monetary2':np.nan})

In [ ]:
# lst2 = ['CompleteIDCertification_sum','EndLoanApply_sum','GetCreditInfo_sum','Login_sum','OpenApp_sum',
#       'StartLoanApply_sum','UseDSRCalc_sum','UseLoanManage_sum','UsePrepayCalc_sum','ViewLoanApplyIntro_sum']
# df = df[lst2]
# df.head(5)

In [ ]:
# df2['monetary2'] = df.sum(axis = 1)
# df2.head(5)

In [ ]:
# train3 = pd.merge(train3, df2, on='user_id', how= 'left')
# train3.head(10)

In [ ]:
# #frequency2 =유저별 visit_rate max
# train3['frequency2'] = train3.groupby('user_id')['visit_rate'].transform(max)
# train3.head(5)


In [ ]:
# #frequency3 =유저별 visit_rate 마지막줄
# #train3['frequency2'] = 

# train3.drop_duplicates(['user_id'], keep = 'last').visit_rate

# #train3.head(5)

# applied_user 변수 만들기

In [38]:
# 한번이라도 apply 했으면 1 아니면 0
train3['applied_user'] = np.where(train3['apply_cnt']==0, 0 , 1)

In [39]:
train3.loc[train3['applied_user']==0,'apply_cnt'].value_counts()

0    777420
Name: apply_cnt, dtype: int64

In [40]:
train3.loc[train3['applied_user']==1,'apply_cnt'].value_counts()

1     458932
2     155232
3      63008
4      29581
5      12896
6       7077
7       3090
8       2077
10       919
9        719
11       707
13       337
12       262
14       127
15       102
16        72
Name: apply_cnt, dtype: int64

In [41]:
train3.shape

(1512558, 71)

In [42]:
train3.isna().sum().sum()

0

In [43]:
# 유저별로 한줄만 남기기
final = train3.drop_duplicates(['user_id'], keep = 'last')
print(final.shape)

(82376, 71)


# csv 내보내기

In [44]:
final.to_csv(r'C:/Users/cny13/OneDrive/바탕 화면/빅콘/segmentation용 데이터/segmentation.csv',index=False)   